In [1]:
import pandas as pd
import numpy as np

In [2]:
filename = './positiveSamples/MI.csv'
savename = './test/AP'

In [3]:
# TODO: 
# 1. Choose better negative sample (ppl with no complications)
# 2. Try different complications and subset broader
# 3. Subset patients with less than 4 visits
# 4. Preprocess Lab results earlier

In [4]:
Lab = pd.read_csv("FullData/LabReduced.csv")
Exam = pd.read_csv("FullData/Exam.csv", usecols = ['Patient_ID', 'Exam1', 'Result1_calc', 'Exam2', 'Result2_calc', 'UnitOfMeasure_calc', 'DateCreated'])
DiseaseCase = pd.read_csv("FullData/DiseaseCase.csv")

/Users/davidhuang/miniforge3/envs/tfARM/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [5]:
# Filtering and processing
# 1. Combine lab and exam sheets
# 2. Lab by 18 desired tests
# 3. Coerce lab & exam values to float
# 4. Remove null for date performed 
# 5. Interquartile ranges
# 6. Datetime

# 7. Disease period range
# 8. Remove redundent visits
# 9. 3 < n < 51


In [6]:
# 1. Combine lab and exam sheets
# Filter exam sheet
# Perhaps include blood pressure
Exam1 = Exam.drop(columns=['Exam2','Result2_calc']).rename(columns={'Exam1':'Name_calc','Result1_calc':'TestResult_calc'})
Exam2 = Exam.drop(columns=['Exam1','Result1_calc']).rename(columns={'Exam2':'Name_calc','Result2_calc':'TestResult_calc'})
Exam = pd.concat([Exam1,Exam2]).dropna()
print(Exam['Name_calc'].value_counts(ascending=False))
Exam = Exam.rename(columns={'DateCreated':'PerformedDate'})
Exam = Exam[Exam.columns[[0,4,1,2,3]]]
Exam = Exam.drop_duplicates(subset=['Patient_ID','PerformedDate'])
print(Exam['Name_calc'].value_counts())
Exam.head()

sBP (mmHg)                  2570192
dBP (mmHg)                  2570192
Weight (kg)                 1102589
BMI (kg/m^2)                 910459
Height (cm)                  736942
Waist Circumference (cm)     115353
PEFR (L/min)                     35
Name: Name_calc, dtype: int64
sBP (mmHg)                  1819054
Weight (kg)                  351314
BMI (kg/m^2)                 309871
Height (cm)                  178870
Waist Circumference (cm)      19442
PEFR (L/min)                     14
Name: Name_calc, dtype: int64


,Patient_ID,PerformedDate,Name_calc,TestResult_calc,UnitOfMeasure_calc
0,4000000368847,2014-02-25,BMI (kg/m^2),38.0,kg/m^2
1,4000000368847,2019-03-18,Height (cm),152.0,cm
2,4000000368847,2019-03-04,sBP (mmHg),139.0,mmHg
3,4000000368847,2018-12-18,sBP (mmHg),142.0,mmHg
4,4000000368847,2016-01-18,sBP (mmHg),146.0,mmHg


In [7]:
lab = pd.concat([Lab,Exam])

In [8]:
# 2 subset by desired tests
tests = ['Alanine Aminotransferase in Serum or Plasma (ALT)',
              'Aspartate Aminotransferase in Serum or Plasma (AST)',
              'Urea (mM)',
              'Plasma Creatinine Clearance (eGFR)',
              'Potassium (mM)',
              'Sodium (mM)',
              'Gamma Glutamyl Transferase in Serum or Plasma (GGT)',
              'Leukocytes in Blood (WBC)',
              'Protein, total (g/L)',
              'Total Cholesterol (TCh)',
              'Total Bilirubin (TBIL)',
              'Platelets in Blood (PLT)',
              'Fasting Blood Glucose (FBG)',
              'Hemoglobin (Hb)',
              'Erythrocytes in Blood (RBC)',
              'Albumin (g/L)',
              'Hemoglobin A1c (HbA1c)',
              'BMI (kg/m^2)']
lab = lab.loc[lab.Name_calc.isin(tests)]
lab['Name_calc'].value_counts()

Hemoglobin A1c (HbA1c)                                 1519423
Hemoglobin (Hb)                                        1469935
Leukocytes in Blood (WBC)                              1220813
Platelets in Blood (PLT)                               1187888
Plasma Creatinine Clearance (eGFR)                     1162456
Erythrocytes in Blood (RBC)                            1115261
Total Cholesterol (TCh)                                 943002
Alanine Aminotransferase in Serum or Plasma (ALT)       903168
Fasting Blood Glucose (FBG)                             808110
Aspartate Aminotransferase in Serum or Plasma (AST)     318870
BMI (kg/m^2)                                            309871
Total Bilirubin (TBIL)                                  295910
Gamma Glutamyl Transferase in Serum or Plasma (GGT)     175251
Sodium (mM)                                             142578
Potassium (mM)                                          136001
Urea (mM)                                              

In [9]:
# 3. Coerce and remove null
# Moved coercion of Test results to floats up here from combined cell
print(lab['TestResult_calc'].isna().sum())
lab['TestResult_calc'] = pd.to_numeric(lab.TestResult_calc,errors='coerce')
print(lab['TestResult_calc'].isna().sum())
lab = lab.dropna(subset=['TestResult_calc'])
print(lab['TestResult_calc'].isna().sum())
print(len(lab.index))

0
81447
0
11743304


In [10]:
# 4. Remove null for date performed
lab = lab.dropna(subset=['PerformedDate'])

In [11]:
# 5. IQR
for test in tests:
    q1 = lab.loc[lab.Name_calc == test].TestResult_calc.quantile(.25)
    q3 = lab.loc[lab.Name_calc == test].TestResult_calc.quantile(.75)
    rng = q3 - q1
    badIndices = lab.loc[(lab.Name_calc == test) & ((lab.TestResult_calc < (q1 - rng)) 
                                    |(lab.TestResult_calc > (q3 + rng)))].index
    lab = lab.drop(badIndices)

In [12]:
len(lab.index)

10491328

In [13]:
# 6. Datetime
lab['PerformedDate'] = pd.to_datetime(lab.PerformedDate,format='%Y-%m-%d')

In [14]:
disease_periods = pd.read_csv(filename, index_col=None)
disease_periods = disease_periods.drop(disease_periods.columns[0], axis=1)
disease_periods = disease_periods.iloc[0:1902]
patient_records = lab.loc[lab['Patient_ID'].isin(disease_periods['Patient_ID'])]
patient_records = pd.merge(patient_records, disease_periods, on='Patient_ID')
print(len(patient_records.index),patient_records.columns)
patient_records.head()

90208 Index(['Patient_ID', 'PerformedDate', 'Name_calc', 'Code_calc',
       'TestResult_calc', 'UnitOfMeasure_calc', 'CompOnset', 'DateOfOnset'],
      dtype='object')


,Patient_ID,PerformedDate,Name_calc,Code_calc,TestResult_calc,UnitOfMeasure_calc,CompOnset,DateOfOnset
0,1000100164803,2013-07-13,Total Cholesterol (TCh),14647-2,2.24,mmol/L,2011-06-24,2007-10-29
1,1000100164803,2013-10-05,Plasma Creatinine Clearance (eGFR),12195-4,66.00,mL/min,2011-06-24,2007-10-29
2,1000100164803,2013-07-13,Plasma Creatinine Clearance (eGFR),12195-4,65.00,mL/min,2011-06-24,2007-10-29
3,1000100164803,2013-07-13,Hemoglobin (Hb),718-7,125.00,g/L,2011-06-24,2007-10-29
4,1000100164803,2013-07-13,Platelets in Blood (PLT),777-3,175.00,10^9/L,2011-06-24,2007-10-29


In [15]:
relVisits = patient_records

In [16]:
relVisits.CompOnset = pd.to_datetime(relVisits.CompOnset,format='%Y-%m-%d')
relVisits.DateOfOnset = pd.to_datetime(relVisits.DateOfOnset,format='%Y-%m-%d')

In [17]:
relVisits = patient_records[(patient_records['PerformedDate'] < patient_records['CompOnset']) & (patient_records['PerformedDate'] >= patient_records['DateOfOnset'])]
len(relVisits.index)

40307

In [18]:
pos_Results = relVisits

In [19]:
# ## 7. Remove Redundent Visits later
pos_Results = pos_Results.groupby('Patient_ID').apply(lambda g: g.sort_values(by = 'PerformedDate', ascending =True))
pos_Results = pos_Results.reset_index(drop = True)

rem = []
for index, row in pos_Results.iterrows() :
    temp = row[['Patient_ID', 'Name_calc','TestResult_calc']]
    if index == 0 :
        prev = temp
        continue
    if temp.equals(prev):
        rem.append(index)
    prev = temp
    #print(prev)

print(len(rem))



169


In [20]:
## NEW
pos_Results_u = pos_Results
print(len(pos_Results_u.index))
pos_Results_u = pos_Results_u.drop(rem)
print(len(pos_Results_u.index))
print(len(pos_Results_u['Patient_ID'].unique()))

40307
40138
718


In [21]:
pos_r_final = pos_Results_u[['Patient_ID','PerformedDate']].groupby('Patient_ID').nunique() 
pos_r_final = pos_r_final[(pos_r_final['PerformedDate'] < 51) & (pos_r_final['PerformedDate'] > 3)]
pos_r_final

,PerformedDate
Patient_ID,
1000100001261,5
1000100003646,42
1000100005097,11
1000100006323,7
1000100006823,32
...,...
11113000008561,4
12120070003088,12
12120070018387,8


In [22]:
n = len(pos_r_final.index)
print(n)
posFinal = pos_Results_u[pos_Results_u['Patient_ID'].isin(pos_r_final.index)]
print(len(relVisits.index) / n) # Average results per person
posFinal.head()

542
74.36715867158672


,Patient_ID,PerformedDate,Name_calc,Code_calc,TestResult_calc,UnitOfMeasure_calc,CompOnset,DateOfOnset
0,1000100001261,2006-01-04,Hemoglobin A1c (HbA1c),17856-6,9.00,%,2009-04-15,2005-02-14
1,1000100001261,2006-01-04,Total Cholesterol (TCh),14647-2,4.23,mmol/L,2009-04-15,2005-02-14
2,1000100001261,2006-11-23,Hemoglobin (Hb),718-7,129.00,g/L,2009-04-15,2005-02-14
3,1000100001261,2006-11-23,Total Cholesterol (TCh),14647-2,4.22,mmol/L,2009-04-15,2005-02-14
4,1000100001261,2006-11-23,Hemoglobin A1c (HbA1c),17856-6,8.10,%,2009-04-15,2005-02-14


In [23]:
# Disease Period
dp = (posFinal['CompOnset'] - posFinal['DateOfOnset']).mean()
print(type(dp))
print(dp)

<class 'pandas._libs.tslibs.timedeltas.Timedelta'>
2357 days 22:29:14.375578144


## Negative Sampling
1. Negative Complement
2. Age and Gender Match
3. Visit Subsetting

In [24]:
# Negative sample part 1
# Import patients with complications
compIDs = pd.read_csv('compIDs.csv')
compIDs = compIDs['0']

# Negative Complement and demographic matching
from random import sample

PatientInfo = pd.read_csv('FullData/Patient.csv')

all_neg_patients = lab.loc[~lab.Patient_ID.isin(compIDs)].Patient_ID.unique()

pos_patients = posFinal['Patient_ID'].unique()
neg_patients = pd.Series([], dtype = 'int64')
size = len(pos_patients)
#takes 45 sec ish to run
for i in range(size):
    refPatient_ID = pos_patients[i]
    refYOB = PatientInfo.loc[PatientInfo.Patient_ID == refPatient_ID].BirthYear.iloc[0]
    refSex = PatientInfo.loc[PatientInfo.Patient_ID == refPatient_ID].Sex.iloc[0]
    PossiblePatients = PatientInfo.loc[PatientInfo.Patient_ID.isin(all_neg_patients) 
                                       & (PatientInfo.Sex == refSex)]
    selectedPat = False
    yearGap = 0
    while not selectedPat:
        #print(len(PossiblePatients))
        possible = PossiblePatients.loc[(PossiblePatients.BirthYear >= refYOB - yearGap) 
                                        & (PossiblePatients.BirthYear <= refYOB + yearGap)].Patient_ID
        if len(possible > 1):
            samp = possible.sample(2)
            selectedPat = samp.iloc[0]
            selectedPat2 = samp.iloc[1]
            neg_patients = neg_patients.append(pd.Series([selectedPat], dtype = 'int64'), ignore_index = True)
            all_neg_patients = all_neg_patients[all_neg_patients != selectedPat]
            neg_patients = neg_patients.append(pd.Series([selectedPat2], dtype = 'int64'), ignore_index = True)
            all_neg_patients = all_neg_patients[all_neg_patients != selectedPat2]

            yearGap += 1
        else :
            print('inf loop')

neg_patients.head()

0    10001000036657
1     8080360171922
2    10001000196144
3     4000000633795
4     4000000256463
dtype: int64

In [25]:
len(neg_patients.index)

1084

In [26]:
# TO DO
# 1. diseaseperiod
# 2. Redundant Visits
# 3. 3 < n < 51

In [27]:
# DiseaseCase
DiseaseCase['Disease'] = DiseaseCase['Disease'].replace("Diabetes Mellitus (ml)","Diabetes Mellitus") # Make all DM2 diagnosis' the same
DiseaseCase['DateOfOnset'] = pd.to_datetime(DiseaseCase.DateOfOnset,format='%Y-%m-%d')
DiseaseCase = DiseaseCase.sort_values(by='DateOfOnset', ascending = True)
DiseaseCase = DiseaseCase.drop_duplicates(subset = ['Patient_ID'])

In [28]:
neg_rec = lab.loc[lab['Patient_ID'].isin(neg_patients)]
neg_rec = pd.merge(neg_rec, DiseaseCase[['Patient_ID','DateOfOnset']], on='Patient_ID')

neg_rec.DateOfOnset = pd.to_datetime(neg_rec.DateOfOnset,format='%Y-%m-%d')

print(len(neg_rec.index)/ len(neg_patients))
neg_rec = neg_rec[neg_rec['PerformedDate'] > neg_rec['DateOfOnset']]
print(len(neg_rec.index)/ len(neg_patients))
print(neg_rec.dtypes)
neg_rec = neg_rec[(neg_rec['PerformedDate'] > neg_rec['DateOfOnset']) & (neg_rec['PerformedDate'] < neg_rec['DateOfOnset']+dp) ]
print(len(neg_rec.index)/ len(neg_patients))

77.75184501845018
59.00092250922509
Patient_ID                     int64
PerformedDate         datetime64[ns]
Name_calc                     object
Code_calc                     object
TestResult_calc              float64
UnitOfMeasure_calc            object
DateOfOnset           datetime64[ns]
dtype: object
45.4880073800738


In [29]:
# ## 7. Remove Redundent Visits later
neg_rec = neg_rec.groupby('Patient_ID').apply(lambda g: g.sort_values(by = 'PerformedDate', ascending =True))
neg_rec = neg_rec.reset_index(drop = True)

rem = []
for index, row in neg_rec.iterrows() :
    temp = row[['Patient_ID', 'Name_calc','TestResult_calc']]
    if index == 0 :
        prev = temp
        continue
    if temp.equals(prev):
        rem.append(index)
    prev = temp
    #print(prev)

print(len(rem))



811


In [30]:
## NEW
neg_rec_u = neg_rec
print(len(neg_rec_u.index))
neg_rec_u = neg_rec_u.drop(rem)
print(len(neg_rec_u.index))
print(len(neg_rec_u['Patient_ID'].unique()))

49309
48498
997


In [31]:
neg_r_final = neg_rec_u[['Patient_ID','PerformedDate']].groupby('Patient_ID').nunique() 
neg_r_final = neg_r_final[(neg_r_final['PerformedDate'] < 51) & (neg_r_final['PerformedDate'] > 3)]
neg_r_final

,PerformedDate
Patient_ID,
1000100001636,5
1000100027105,5
1000100031285,5
1000100040340,13
1000100042774,4
...,...
12120090042387,20
12120090044214,14
12120090045154,17


In [32]:
negFinal = neg_rec_u[neg_rec_u['Patient_ID'].isin(neg_r_final.index)]
negIDs = negFinal['Patient_ID'].unique()
negFinal.head()

,Patient_ID,PerformedDate,Name_calc,Code_calc,TestResult_calc,UnitOfMeasure_calc,DateOfOnset
0,1000100001636,2011-04-14,Erythrocytes in Blood (RBC),789-8,4.97,10^12/L,2010-03-25
1,1000100001636,2011-04-14,Hemoglobin A1c (HbA1c),17856-6,5.60,%,2010-03-25
2,1000100001636,2011-04-14,Gamma Glutamyl Transferase in Serum or Plasma ...,2324-2,23.00,U/L,2010-03-25
3,1000100001636,2011-04-14,Alanine Aminotransferase in Serum or Plasma (ALT),1742-6,19.00,U/L,2010-03-25
4,1000100001636,2011-04-14,Platelets in Blood (PLT),777-3,243.00,10^9/L,2010-03-25


In [33]:
Results = pd.concat([posFinal, negFinal])

In [34]:
turtiles = pd.read_csv('TestTurtiles.csv')

In [35]:
for index, row in Results.iterrows() :
    test = row['Name_calc']
    value = row['TestResult_calc']
    #feature = [0,0,0]
    t1 = float(turtiles.loc[turtiles['TestName'] == test]['t1'])
    t2 = float(turtiles.loc[turtiles['TestName'] == test]['t2'])
    #print(t1,t2,value)
    if value <= t1 :
        feature = 1.0 #[1,0,0]
    elif value >= t2 :
        feature = 3.0 #[0,0,1]
    else :
        feature = 2.0 #[0,1,0]
    Results.at[index, 'TestResult_calc'] = feature
Results.head()      

,Patient_ID,PerformedDate,Name_calc,Code_calc,TestResult_calc,UnitOfMeasure_calc,CompOnset,DateOfOnset
0,1000100001261,2006-01-04,Hemoglobin A1c (HbA1c),17856-6,3.0,%,2009-04-15,2005-02-14
1,1000100001261,2006-01-04,Total Cholesterol (TCh),14647-2,1.0,mmol/L,2009-04-15,2005-02-14
2,1000100001261,2006-11-23,Hemoglobin (Hb),718-7,1.0,g/L,2009-04-15,2005-02-14
3,1000100001261,2006-11-23,Total Cholesterol (TCh),14647-2,2.0,mmol/L,2009-04-15,2005-02-14
4,1000100001261,2006-11-23,Hemoglobin A1c (HbA1c),17856-6,2.0,%,2009-04-15,2005-02-14


In [36]:
Results.groupby('TestResult_calc').agg('count')

,Patient_ID,PerformedDate,Name_calc,Code_calc,UnitOfMeasure_calc,CompOnset,DateOfOnset
TestResult_calc,,,,,,,
1.0,27709,27709,27709,26982,27665,11774,27709
2.0,24404,24404,24404,23658,24376,10321,24404
3.0,24797,24797,24797,24117,24754,10335,24797


In [37]:
Results = Results.groupby('Patient_ID').apply(lambda g: g.sort_values(by = 'PerformedDate', ascending =True))
Results = Results.reset_index(drop = True)
rem = []
for index, row in Results.iterrows() :
    temp = row[['Patient_ID', 'Name_calc','TestResult_calc']]
    if index == 0 :
        prev = temp
        continue
    if temp.equals(prev):
        rem.append(index)
    prev = temp
    #print(prev)

print(len(rem))


2168


In [38]:
Results_u = Results
print(len(Results_u.index))
Results_u = Results_u.drop(rem)
print(len(Results_u.index))

76910
74742


In [39]:
AllVisits = {}
VisitValues = {}
for index, row in Results_u.iterrows() :
    temp = (str(row['Patient_ID']),str(row['PerformedDate']))
    diag = (row['Name_calc'],row['TestResult_calc'])
    if temp not in AllVisits.keys():
        AllVisits[temp] = []
    AllVisits[temp] = AllVisits[temp] + [diag]

PVDF = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in AllVisits.items() ]))
PVDF = PVDF.transpose()
PVDF = PVDF.dropna(axis=0,how='all') # Remove NA


In [40]:
eTests = []
for t in tests :
    for i in range(1,4) :
        eTests.append(t+str(i))

In [41]:
RowFeatures = pd.DataFrame(data = 0,
                       index = PVDF.index,
                       columns=eTests)
RowFeatures

Alanine Aminotransferase in Serum or Plasma (ALT)1  \
1000100001261  2006-01-04 00:00:00                                                  0    
               2006-11-23 00:00:00                                                  0    
               2008-01-03 00:00:00                                                  0    
               2008-04-23 00:00:00                                                  0    
               2008-12-30 00:00:00                                                  0    
...                                                                               ...    
12120120055201 2017-10-02 00:00:00                                                  0    
               2018-01-04 00:00:00                                                  0    
               2018-01-18 00:00:00                                                  0    
               2018-06-08 00:00:00                                                  0    
               2018-11-27 00:00:00                                                  0    

                                    Alanine Aminotransferase in Serum or Plasma (ALT)2  \
1000100001261  2006-01-04 00:00:00                                                  0    
               2006-11-23 00:00:00                                                  0    
               2008-01-03 00:00:00                                                  0    
               2008-04-23 00:00:00                                                  0    
               2008-12-30 00:00:00                                                  0    
...                                                                               ...    
12120120055201 2017-10-02 00:00:00                                                  0    
               2018-01-04 00:00:00                                                  0    
               2018-01-18 00:00:00                                                  0    
               2018-06-08 00:00:00                                                  0    
               2018-11-27 00:00:00                                                  0    

                                    Alanine Aminotransferase in Serum or Plasma (ALT)3  \
1000100001261  2006-01-04 00:00:00                                                  0    
               2006-11-23 00:00:00                                                  0    
               2008-01-03 00:00:00                                                  0    
               2008-04-23 00:00:00                                                  0    
               2008-12-30 00:00:00                                                  0    
...                                                                               ...    
12120120055201 2017-10-02 00:00:00                                                  0    
               2018-01-04 00:00:00                                                  0    
               2018-01-18 00:00:00                                                  0    
               2018-06-08 00:00:00                                                  0    
               2018-11-27 00:00:00                                                  0    

                                    Aspartate Aminotransferase in Serum or Plasma (AST)1  \
1000100001261  2006-01-04 00:00:00                                                  0      
               2006-11-23 00:00:00                                                  0      
               2008-01-03 00:00:00                                                  0      
               2008-04-23 00:00:00                                                  0      
               2008-12-30 00:00:00                                                  0      
...                                                                               ...      
12120120055201 2017-10-02 00:00:00                                                  0      
               2018-01-04 00:00:00                       

In [42]:
for index, row in RowFeatures.iterrows() :
    temp = PVDF.loc[index].dropna()
    for t in temp :
        test = t[0] + str(int(t[1]))
        row[test] = 1
RowFeatures

Alanine Aminotransferase in Serum or Plasma (ALT)1  \
1000100001261  2006-01-04 00:00:00                                                  0    
               2006-11-23 00:00:00                                                  0    
               2008-01-03 00:00:00                                                  0    
               2008-04-23 00:00:00                                                  0    
               2008-12-30 00:00:00                                                  0    
...                                                                               ...    
12120120055201 2017-10-02 00:00:00                                                  0    
               2018-01-04 00:00:00                                                  0    
               2018-01-18 00:00:00                                                  0    
               2018-06-08 00:00:00                                                  0    
               2018-11-27 00:00:00                                                  0    

                                    Alanine Aminotransferase in Serum or Plasma (ALT)2  \
1000100001261  2006-01-04 00:00:00                                                  0    
               2006-11-23 00:00:00                                                  0    
               2008-01-03 00:00:00                                                  0    
               2008-04-23 00:00:00                                                  0    
               2008-12-30 00:00:00                                                  0    
...                                                                               ...    
12120120055201 2017-10-02 00:00:00                                                  0    
               2018-01-04 00:00:00                                                  0    
               2018-01-18 00:00:00                                                  0    
               2018-06-08 00:00:00                                                  0    
               2018-11-27 00:00:00                                                  0    

                                    Alanine Aminotransferase in Serum or Plasma (ALT)3  \
1000100001261  2006-01-04 00:00:00                                                  0    
               2006-11-23 00:00:00                                                  0    
               2008-01-03 00:00:00                                                  1    
               2008-04-23 00:00:00                                                  0    
               2008-12-30 00:00:00                                                  1    
...                                                                               ...    
12120120055201 2017-10-02 00:00:00                                                  0    
               2018-01-04 00:00:00                                                  0    
               2018-01-18 00:00:00                                                  1    
               2018-06-08 00:00:00                                                  0    
               2018-11-27 00:00:00                                                  0    

                                    Aspartate Aminotransferase in Serum or Plasma (AST)1  \
1000100001261  2006-01-04 00:00:00                                                  0      
               2006-11-23 00:00:00                                                  0      
               2008-01-03 00:00:00                                                  0      
               2008-04-23 00:00:00                                                  0      
               2008-12-30 00:00:00                                                  0      
...                                                                               ...      
12120120055201 2017-10-02 00:00:00                                                  0      
               2018-01-04 00:00:00                       

In [43]:
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components = 18)
visits_emb = svd.fit_transform(RowFeatures)
reducedDF = pd.DataFrame(data=visits_emb, index=RowFeatures.index)

In [44]:
IDs = negIDs

In [45]:
VisitHistorySVD = {}
for index, row in reducedDF.iterrows() :
    temp = int(index[0])
    featureVector = list(row)
    if temp not in VisitHistorySVD.keys():
        if temp in IDs :
            VisitHistorySVD[temp] = [0]
        else :
            VisitHistorySVD[temp] = [1]
    VisitHistorySVD[temp] = VisitHistorySVD[temp] + featureVector




In [46]:
m = [0]
for k,v in VisitHistorySVD.items() :
    temp = len(v)
    if temp > m[-1] :
        m.append(k)
        m.append(temp)
        
print(m)

[0, 1000100001261, 91, 1000100003646, 757, 1000100033226, 793, 1000100043818, 829, 1000100067326, 901]


In [47]:
# Fill in to 900
for k,v in VisitHistorySVD.items() :
    fillLen = 901 - len(v)
    VisitHistorySVD[k] = v + [0]*fillLen

In [48]:
# MODELING
# Convert data to x y
x, y = [], []
for k,v in VisitHistorySVD.items() :
    if v[0] == 1 :
        y.append([1,0])
    if v[0] == 0 :
        y.append([0,1])
    x.append(v[1:])

print(len(x), len(y))

1332 1332


In [49]:
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Embedding,GRU, LSTM, GlobalMaxPooling1D, SpatialDropout1D, GRU, Bidirectional, Input, TimeDistributed, Reshape

In [50]:
# Split data
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.10, shuffle = True)
print(len(x_train), 'training')
print(len(x_test), 'testing')
#print(x_test[:10], y_test[:10])

1198 training
134 testing
[[1.4137572319240632, 0.5783694618648614, -0.21730711823254273, -0.22727443896534746, -0.5979089129817736, -1.0270113987335445, 0.51156745383495, -0.13817849194664494, 0.3973063950860568, -0.35690149450156294, -0.8661720387623594, 0.6362413209605513, -0.07437445625145944, -0.0674098138267061, 0.46059049687946946, 0.5150468429532188, 0.2536484090240936, -0.2759355090359942, 0.00350292897181061, -0.002834516079639181, 0.00027611801921787214, -0.0009776996762047884, -0.0003762821930973063, -0.00012721986070730277, 0.0009403173289827176, -4.0731230543511426e-05, 6.932141170473089e-05, -0.00023443794194515725, -0.0012289827698089236, 0.000563185833312005, -0.0025198124822651768, 0.000638396623282752, 0.0021401169188752546, -0.0026214928388869738, -0.0015498304467376298, -0.0008787377044002959, 0.5042936355271272, 0.6813260329141511, -0.27993757799256375, -0.5283855646144816, 0.19736656482768966, -0.09613767278095513, 0.186798981279112, 0.0878205915207011, 0.0088038

In [51]:
#x_test[0]

In [52]:
# Bidirectional GRU

# Random search drop out rate and number of units/neurons
x_train = np.array(x_train)
x_test = np.array(x_test)
input_shape = (x_train.shape[1], 1) #900,1

units = 100


model = Sequential()
model.add(Reshape((50, 18), input_shape=(900,)))
# 1 sees the past 
# 2 sees the past and the future


dropout = 0.2 # for regularization
print('Number of hidden units: ', units, 'Dropout: ', dropout)
model.add(Bidirectional(GRU(units, input_shape=input_shape))) # 64, $128$, 256, 512
model.add(Dropout(dropout))
model.add(Dense(2, activation='softmax'))

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)
print('Training...')
history = model.fit(
    np.array(x_train),
    np.array(y_train),
    batch_size = 128, 
    epochs = 25,
    validation_split = 0.10
)




In [53]:
print(model.evaluate(x_test, np.array(y_test)))
preds = model.predict(x_test)

In [54]:
model.save(savename)

In [55]:
from matplotlib import pyplot as plt
plt.clf()
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'g', label='Training loss')
plt.plot(epochs, val_loss, 'y', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [56]:
from sklearn.metrics import average_precision_score
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, preds)

In [57]:
# 70s val accuracy, 50-low60s test accuracy LSTM BOOHOO
# 67 val accuracy, 73% accuracy, auc_roc 0.836 GRU
# 0.7426 val accuracy, 0.7815 accuracy, roc 0.8745557924662403 GRU

In [58]:
# TODOS : 
# 1. Check features
# 2. Try without SVD
# 3. Try with different diseases
# 4. 